# Assignment 1
## Digital Musicology

@authors: Joris Monnet, Xingyu Pan, Yutaka Osaki, Yiwei Liu

Due Date: 24/04/2024


In [3]:
import music21

# Task B

From your analysis in task A, think about how to model these aspects computationally. In this task, choose one or more aspect(s) in your analysis you would like to model.

You should write an algorithm that outputs a MIDI file from an musicXML or the unperformed MIDI. In your report, please clearly describe your model.

In [210]:
midi_file_path = 'corrected_midi_score.mid'
score = music21.converter.parse(midi_file_path)

In [ ]:

def print_durations(score, part_index, measure_number):
    part = score.parts[part_index]
    measure = part.measure(measure_number)
    notes = measure.notes
    print(f"Track {part_index}, Measure {measure_number}, Durations:")
    for note in notes:
        if isinstance(note, music21.note.Note):
            print(f"Note {note.pitch}: {note.duration.quarterLength} quarter lengths")
        elif isinstance(note, music21.chord.Chord):
            pitches = ', '.join(str(p) for p in note.pitches)
            print(f"Chord {pitches}: {note.duration.quarterLength} quarter lengths")


* 1st ~ 14th 

In [250]:
from music21 import midi

def adjust_durations_for_specific_measure(score, measure_number, track1_new_durations=[0.75, 0.4, 0.3, 0.3, 1.25]):
    """
    Adjusts the durations of notes in a specific measure of a score and ensures that the durations
    of notes in another track are updated proportionally to match the total duration of the modified measure.

    Parameters:
        score (music21.stream.Score): The score containing the music parts.
        measure_number (int): The measure number to adjust durations for.
        track1_new_durations (list[float]): List of new durations for notes in track1.

    Returns:
        music21.stream.Score: The score with adjusted durations.
    """
    # Get the relevant tracks from the score
    track1 = score.parts[1]
    track0 = score.parts[0]

    # Get the specific measure from each track
    track1_measure = track1.measure(measure_number)
    track0_measure = track0.measure(measure_number)

    # Update durations of notes in track1
    for note, new_duration in zip(track1_measure.notes, track1_new_durations):
        note.duration.quarterLength = new_duration

    # Calculate total new duration of the modified track1 measure
    total_new_duration = sum(track1_new_durations)

    # Update durations of notes in track0 proportionally
    track0_notes = track0_measure.notes
    total_original_duration = sum(note.duration.quarterLength for note in track0_notes)
    scale_factor = total_new_duration / total_original_duration if total_original_duration != 0 else 0

    for note in track0_notes:
        note.duration.quarterLength *= scale_factor

    return score

def execute_adjust_durations_for_specific_measure(score, start_measure_number, end_measure_number):
    # Adjust durations from start_measure_number to end_measure_number
    for i in range(start_measure_number, end_measure_number+1):
        score = adjust_durations_for_specific_measure(score, i)
        print(f"Measure {i} adjusted.")

    return score


* 15 ~ 18

In [248]:
from music21 import midi, stream

def change_duration_in_measure(score, measure_number, target_duration, new_duration, track_number=0):
    """
    Changes the duration of notes with a specific duration in a specified measure of a specified track within a MIDI file.

    Params:
        midi_file_path (str): Path to the MIDI file to be processed.
        track_number (int): The track number to process (0-indexed).
        measure_number (int): The measure number where notes' duration will be changed (1-indexed).
        target_duration (float): The duration of the notes to be changed.
        new_duration (float): The new duration to be applied to the notes.

    Returns:
        music21.stream.Score: The modified Score object with updated note durations in the specified measure.
    """
    # Access the specific part (track)
    target_part = score.parts[track_number]
    
    # Access the specific measure
    target_measure = target_part.measure(measure_number)
    # Iterate over all notes in the measure
    for note in target_measure.notes:
        # Check if the note's duration matches the target duration
        if note.duration.quarterLength == target_duration:
            # Change the note's duration to the new specified duration
            note.duration.quarterLength = new_duration

    return score
def execute_change_duration_in_measure(score, start_measure_number, end_measure_number):
    # Adjust durations from start_measure_number to end_measure_number
    for i in range(start_measure_number, end_measure_number+1):
        score = change_duration_in_measure(score, 15, 0.5, 0.3, 0)
        print(f"Measure {i} adjusted.")

    return score

* 19 ~ 26

In [244]:
from music21 import midi

def accelerate_measure(score, measure_number, accelerate_rate, track_numbers=[0, 1]):
    """
    Accelerates the durations of notes within a specified measure by adjusting their lengths relative to the first note's duration, such that the last note's duration is accelerate_rate times faster than the first note's duration.

    Parameters:
        score (music21.stream.Score): The score object containing the measure to be accelerated.
        measure_number (int): The number of the measure to be accelerated.
        accelerate_rate (float): The rate at which the durations should accelerate.
            means the last note's duration will be half of the first note's duration.
        track_numbers (list of int): List of track numbers to process (0 or 1).

    Returns:
        music21.stream.Score: The modified score object with the accelerated measure.
    """
    for track_number in track_numbers:
        track = score.parts[track_number]
        measure = track.measure(measure_number)
        if measure is None:
            continue
        notes = measure.recurse().notes
        base_duration = notes[0].duration.quarterLength
        num_notes = len(notes)
        for i, note in enumerate(notes):
            if i == 0:
                continue
            accelerate_factor = accelerate_rate ** ((i + 1) / num_notes)
            note.duration.quarterLength = base_duration / accelerate_factor
    
    return score

* 27 ~ 30

In [247]:
def increase_volume_of_higher_notes_in_track(score, start_measure_number, end_measure_number, track_number=0, volume_increase=10):
    """
    Increases the volume of the higher-pitched note in pairs of sixteenth notes within specified measures in a specific track of a score.
    
    Params:
        score (music21.stream.Score): The Score object to be processed.
        track_number (int): The track number to process (1-indexed).
        start_measure_number (int): The measure number to start processing (inclusive).
        end_measure_number (int): The measure number to end processing (inclusive).
        volume_increase (int): The amount by which to increase the volume of the higher-pitched note.

    Returns:
        music21.stream.Score: The modified Score object with increased volumes for higher-pitched notes in the specified measures of the specified track.
    """
    
    target_part = score.parts[track_number]
    
    for i in range(start_measure_number, end_measure_number + 1):
        target_measure = target_part.measure(i)
        for note1, note2 in zip(target_measure.notes[:-1], target_measure.notes[1:]):
            if note1.duration.quarterLength == 0.25 and note2.duration.quarterLength == 0.25:
                # Extract pitches; handle both Note and Chord objects
                pitch1 = note1.pitches[-1] if isinstance(note1, music21.chord.Chord) else note1.pitch
                pitch2 = note2.pitches[-1] if isinstance(note2, music21.chord.Chord) else note2.pitch

                # Increase volume of the higher-pitched note
                if pitch1 > pitch2:
                    note1.volume.velocity += volume_increase
                else:
                    note2.volume.velocity += volume_increase
            
        print(f"Measure {i} adjusted.")

    return score

### Execute all function

In [253]:

# Load the MIDI file
mf = midi.MidiFile()
mf.open('corrected_midi_score.mid')
mf.read()
mf.close()

# Convert MIDI file to a music21 stream
score = midi.translate.midiFileToStream(mf)
modified_score = execute_adjust_durations_for_specific_measure(score, 1, 14)
modified_score = execute_change_duration_in_measure(modified_score, 15, 18)
modified_score = accelerate_measure(modified_score, 19, 1.2)
modified_score = accelerate_measure(modified_score, 20, 0.9)
modified_score = accelerate_measure(modified_score, 21, 1.2)
modified_score = accelerate_measure(modified_score, 22, 0.8)
modified_score = accelerate_measure(modified_score, 23, 1.0)
modified_score = accelerate_measure(modified_score, 24, 1.0)
modified_score = accelerate_measure(modified_score, 25, 1.2)
modified_score = accelerate_measure(modified_score, 26, 1.0)
modified_score = increase_volume_of_higher_notes_in_track(modified_score, 27, 30)

Measure 1 adjusted.
Measure 2 adjusted.
Measure 3 adjusted.
Measure 4 adjusted.
Measure 5 adjusted.
Measure 6 adjusted.
Measure 7 adjusted.
Measure 8 adjusted.
Measure 9 adjusted.
Measure 10 adjusted.
Measure 11 adjusted.
Measure 12 adjusted.
Measure 13 adjusted.
Measure 14 adjusted.
Measure 15 adjusted.
Measure 16 adjusted.
Measure 17 adjusted.
Measure 18 adjusted.
Measure 27 adjusted.
Measure 28 adjusted.
Measure 29 adjusted.
Measure 30 adjusted.


### others

In [207]:
import music21  # music21 ライブラリをインポート

def count_notes_in_measure(score, measure_number):
    """
    指定された小節番号における全パートの音符の数を表示する関数。

    Parameters:
        score (music21.stream.Score): 解析する楽譜。
        measure_number (int): 音符の数を数えたい小節番号。
    """
    # スコアの各パートをループ処理
    for part_index, part in enumerate(score.parts):
        measure = part.measure(measure_number)  # 指定した小節番号の小節を取得
        notes = measure.notes  # 小節内の音符リストを取得
        print(f"Part {part_index + 1}: {len(notes)} notes in measure {measure_number}")

# MIDI ファイルを読み込む
midi_file_path = 'corrected_midi_score.mid'
score = music21.converter.parse(midi_file_path)

# 例として、小節番号 5 の音符の数を調べる
count_notes_in_measure(score, 14)


Part 1: 0 notes in measure 14
Part 2: 5 notes in measure 14


In [233]:
from music21 import midi

# Load the MIDI file
mf = midi.MidiFile()
mf.open('corrected_midi_score.mid')
mf.read()
mf.close()

# Convert MIDI file to a music21 stream
my_stream = midi.translate.midiFileToStream(mf)

# Select a specific track, for example track 1
specific_track = my_stream.parts[0]  # This assumes that the part you are interested in is the first part
i = 19
# Select a specific measure from the track, for example measure 5
specific_measure = specific_track.measure(i)  # This gets the 5th measure of the selected part

# Count the number of notes in the specific measure
note_count = len(specific_measure.recurse().notes)
print(f"Number of notes in measure {i} of track 1:", note_count)

Number of notes in measure 19 of track 1: 13


## Task C

Compare your generated MIDI with both the unperformed MIDI and the human performace recording. Discuss their differences and potentially what and how you can further improve. This should be part of your discussion section in the report.

In [ ]:
# TODO visualizations of differences ?